In [95]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from random import seed, shuffle
import os
from datetime import datetime

from pyannote.audio import Pipeline
from pyannote.core import json

from matplotlib import pyplot as plt, lines
import seaborn as sns

from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import StandardScaler

from scipy import stats
from scipy.spatial.distance import directed_hausdorff, euclidean, cosine, pdist

from src import (
    downsample as downsamp_audio, 
    embedding_extractor as ee, 
    dimension_reducer as dr, 
    distance_metrics as dm, 
    downsample as ds,
    emb_manipulator as em
)

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
##Downsample audio to 16kHz
root_dir= "/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/053122_new_participants/"
audio_files = os.path.join(root_dir,"audio")

#make a folder with today's date for the downsampled audio
down_sample_dir = os.path.join(root_dir,
                               datetime.now().strftime('%Y_%m_%d') + '_16kHz')

if not os.path.exists(down_sample_dir):
    os.mkdir(down_sample_dir)

In [ ]:
ds.downsamp_audio(audio_files, down_sample_dir)

processing023.wav


In [12]:
##Diarize audio to find when mother is speaking and visualize it to choose label
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

diar_dir = os.path.join(root_dir,'audio',
                           datetime.now().strftime('%Y_%m_%d') + '_diarized')
if not os.path.exists(diar_dir):
    os.mkdir(diar_dir)

In [ ]:
for file in os.listdir(down_sample_dir):
    if file.endswith('wav'):
        diar = pipeline(os.path.join(down_sample_dir,file))
        json.dump_to(diar, os.path.join(diar_dir, os.path.splitext(file)[0]+'.json'))

In [16]:
diar_dir = '/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/053122_new_participants/2022_05_31_diarized'

In [68]:
# Based on output of diarization, manually make speaker map to set which speaker we want
speaker_map = {'021':'SPEAKER_01', '022':'SPEAKER_00', 'BB003':'SPEAKER_00'}

In [73]:
diarized = {os.path.splitext(file)[0]:
            json.load_from(os.path.join(diar_dir,file)).label_timeline(
                speaker_map[os.path.splitext(file)[0]]) 
            for file in os.listdir(diar_dir)}

In [75]:
emb_dir = '/Users/rahulbrito/Documents/projects/infantvoice/data/embeddings'
save_name = datetime.now().strftime('%Y_%m_%d') + '_emb.csv'
pyannote_emb = ee.pyannote_extract_directory(down_sample_dir, diarized,emb_dir,save_name,save=True)

Processing022.wav
Done


In [112]:
files = ['03016_diarized_pyv2.csv','2022_06_01_emb.csv']

emb_unscaled = pd.DataFrame()

for f in files:
    emb_unscaled = pd.concat([emb_unscaled,pd.read_csv(os.path.join(emb_dir,f), index_col=0)], ignore_index=True)

#emb_down_unscaled = em.resample_data(emb,1)

scaler = StandardScaler()
emb = pd.DataFrame(scaler.fit_transform(emb_unscaled.drop(columns='part_id')))
emb['part_id'] = emb_unscaled.part_id.astype('string').to_numpy()


In [ ]:
#default
#learning_rate = emb_down.shape[0]//12
#perplexity = 30

#max global structure
perplexity = 135
learning_rate = 200

#default
#n_neighbors = 10
#dist=0.1

#max global structure
n_neighbors=14
dist=10E-3

###Run dimension reduction with different methods on the embeddings to establish a baseline before leaving any participants out
#Using tsne with perplexity=30 (default) since N/100<30, and learning rate of n/12. Random seed is 42 so the result is the same each time
emb_tsne = dr.run_tsne(emb_down, perplexity = perplexity, init='pca', learning_rate=learning_rate)

#run umap with default parameters
emb_umap = dr.run_umap(emb_down, n_neighbors=n_neighbors, dist=dist)


labels = emb_down.part_id

#PACMAP for comparison
emb_pacmap = dr.run_pacmap(emb_down)